# OpenAI Assistants APIs

The Assistants' API lets you create AI assistants in your applications. These assistants follow instructions and use models, tools, and knowledge to answer user questions. In this notebook we are going to use one of the tools, retriever,
to query against two pdf documents we will upload.

The architeture and data flow diagram below depicts the interaction among all components that comprise OpenAI Assistant APIs. Central to understand is the Threads and Runtime that executes anyschronously, adding and reading messages to the Threads.

For integrating the Assistants API:

1. Creat an Assistant with custom instructions and select a model. Optionally, enable tools like Code Interpreter, Retrieval, and Function Calling.

2. Initiate a Thread for each user conversation.
    
3. Add user queries as Messages to the Thread.

4.  Run the Assistant on the Thread for responses, which automatically utilizes the enabled tools

Below we follow those steps to demonstrate how to integrate Assistants API, using function tool, to ask our Assistant to interact with an external webservice, such
as Google Search. This external service could be any external [API Webserivce](https://apilayer.com/)

The OpenAI documentation describes in details [how Assistants work](https://platform.openai.com/docs/assistants/how-it-works).

<img src="./images/assistant_ai_tools_functions_google.png">


## How to use Assistant API using Tools: Function calling
In this example, we will use an external service. That is,
our function will call an external web service: Google Search API
to fetch the results of the query requested. 

This is an example of how an Assistant can employ an external tool, such as a webservice. Our query could be part of a larger
application using LLM and Assitant to respond to user query, and then using the results fetched to use downstream.

Let's see how we can do it. The steps are not dissimilar to our
previous notebook. The only difference here is that our function is make an external webservice call and we have a different function JSON definition to match the the arguments to our function call, which it can use to pass to an external webservice.

In [1]:
import warnings
import os
import json

import openai
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv
from typing import List, Dict, Any
from assistant_utils import print_thread_messages, upload_files, \
                            loop_until_completed, create_assistant_run
from function_utils import add_prime_numbers
from google_search_utils import google_search

Load our .env file with respective API keys and base url endpoints. Here you can either use OpenAI or Anyscale Endpoints. **Note**: Assistant API calling for Anyscale Endpoints (which serves only OS modles) is not yet aviable).

In [2]:
warnings.filterwarnings('ignore')

_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_base = os.getenv("ANYSCALE_API_BASE", os.getenv("OPENAI_API_BASE"))
openai.api_key = os.getenv("ANYSCALE_API_KEY", os.getenv("OPENAI_API_KEY"))
google_api_key = os.getenv("GOOGLE_API_KEY", "")
MODEL = os.getenv("MODEL")
print(f"Using MODEL={MODEL}; base={openai.api_base}")

Using MODEL=gpt-4-1106-preview; base=https://api.openai.com/v1


In [3]:
from openai import OpenAI

client = OpenAI(
    api_key = openai.api_key,
    base_url = openai.api_base
)

### Step 1: Create our custom function definition
This our JSON object definiton for our function:
* name of the function
* parameters for the funtion
* type of arguments
* descriptions for function and each parameter type

In [4]:
search_google_query = {
    "name": "google_search",
    "description": "A function takes in a search query, api key, and optionly num of results specified. ",
    "parameters": {
        "type": "object",
        "properties": {
            "query" : {
                "type": "string",
                "description" : "The search query to send to the Google Search Engine"
            },
            "api_key": {
                "type": "string",
                "description" : "Google Search API key"

            },
            "num_results" : {
                "type": "integer",
                "description" : "number of results. This is a optional one, default is 1"

            }
        }
    },
    "required": ["query", "api_key"]
}
tools = [{'type': 'function', 'function': search_google_query}]

### Step 2: Create an Assistant 
Before you can start interacting with the Assistant to carry out any tasks, you need an AI assistant object. Supply the Assistant with a model to use, tools, i.e., functions

In [5]:
instructions = """You are a knowledgeable and helpful chatbot trained to resolve Google interact
with external webservices such as Google via help of function calls
"""
assistant = client.beta.assistants.create(name="AI Math Tutor",
                                           instructions=instructions,
                                           model=MODEL,
                                           tools=tools)
assistant

Assistant(id='asst_LQOXrnwz8i83JEiw8pFoofN8', created_at=1703382082, description=None, file_ids=[], instructions='You are a knowledgeable and helpful chatbot trained to resolve Google interact\nwith external webservices such as Google via help of function calls\n', metadata={}, model='gpt-4-1106-preview', name='AI Math Tutor', object='assistant', tools=[ToolFunction(function=FunctionDefinition(name='google_search', description='A function takes in a search query, api key, and optionly num of results specified. ', parameters={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'The search query to send to the Google Search Engine'}, 'api_key': {'type': 'string', 'description': 'Google Search API key'}, 'num_results': {'type': 'integer', 'description': 'number of results. This is a optional one, default is 1'}}}), type='function')])

### Step 3: Create an empty thread 
As the diagram above shows, the Thread is the object with which the AI Assistant runs will interact with, by fetching messages and putting messages to it. Think of a thread as a "conversation session between an Assistant and a user. Threads store Messages and automatically handle truncation to fit content into a model’s context window."

In [6]:
thread = client.beta.threads.create()
thread

Thread(id='thread_X0TJGhkcpuYoCo9OaCEJ5rB8', created_at=1703382083, metadata={}, object='thread')

### Step 4: Add your message query to the thread for the Assistant

In [7]:
message_1 = client.beta.threads.messages.create(
    thread_id=thread.id, 
    role="user",
    content="""Search Google for the top 5 Italian resturants in San Francisco.
    """
)
message_1

ThreadMessage(id='msg_kddbdIejphUxXohBjpDc4OjX', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='Search Google for the top 5 Italian resturants in San Francisco.\n    '), type='text')], created_at=1703382085, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_X0TJGhkcpuYoCo9OaCEJ5rB8')

### Step 5: Create a Run for the Assistant
A Run is an invocation of an Assistant on a Thread. The Assistant uses it’s configuration and the Thread’s Messages to perform tasks by calling models and tools. As part of a Run, the Assistant appends Messages to the Thread.

Note that Assistance will run asychronously: the run has the following
lifecycle and states: [*expired, completed, requires, failed, cancelled*]. Run objects can have multiple statuses.

<img src="https://cdn.openai.com/API/docs/images/diagram-1.png">

In [8]:
instruction_msg = """Top Five best Italian restaurants in San Francisco. Only return
top 5 results
"""
run_1 = create_assistant_run(client, assistant, thread, instruction_msg)
run_1

Run(id='run_iafPlXYsQQLVbQ8csu0HKVlJ', assistant_id='asst_LQOXrnwz8i83JEiw8pFoofN8', cancelled_at=None, completed_at=None, created_at=1703382087, expires_at=1703382687, failed_at=None, file_ids=[], instructions='Top Five best Italian restaurants in San Francisco. Only return\ntop 5 results\n', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_X0TJGhkcpuYoCo9OaCEJ5rB8', tools=[ToolAssistantToolsFunction(function=FunctionDefinition(name='google_search', description='A function takes in a search query, api key, and optionly num of results specified. ', parameters={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'The search query to send to the Google Search Engine'}, 'api_key': {'type': 'string', 'description': 'Google Search API key'}, 'num_results': {'type': 'integer', 'description': 'number of results. This is a optional one, default is 1'}}}), type='functi

### Step 6: Retrieve the status

In [9]:
run_1_obj = client.beta.threads.runs.retrieve(
    thread_id = thread.id,
    run_id = run_1.id
)
print(run_1_obj, run_1_obj.status)

Run(id='run_iafPlXYsQQLVbQ8csu0HKVlJ', assistant_id='asst_LQOXrnwz8i83JEiw8pFoofN8', cancelled_at=None, completed_at=None, created_at=1703382087, expires_at=1703382687, failed_at=None, file_ids=[], instructions='Top Five best Italian restaurants in San Francisco. Only return\ntop 5 results\n', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_NTfY9SxINpIfhc7pt3lJQrz1', function=Function(arguments='{"query":"top 5 Italian restaurants in San Francisco","num_results":5}', name='google_search'), type='function')]), type='submit_tool_outputs'), started_at=1703382087, status='requires_action', thread_id='thread_X0TJGhkcpuYoCo9OaCEJ5rB8', tools=[ToolAssistantToolsFunction(function=FunctionDefinition(name='google_search', description='A function takes in a search query, api key, and optionly num of results specified. ', parameters={

Loop until run status is "required_action," which is a trigger notification to extract arguments generated by the LLM model and carry onto the next step: invoke the function with the generated arguments.

In [10]:
loop_until_completed(client, thread, run_1_obj)

Get the tools output. This where your application can call into
the funtion you want invoked. 

### Step 7: Invoke our custom code function

In [11]:
def get_output_for_tool_call(tool_call: List[Any]) -> object:
    for each_tool in tool_calls:
        tool_call_id = each_tool.id
        function_args = json.loads(each_tool.function.arguments)
        print(f"Tool ID: {tool_call_id}")
        print(f"Function args:{function_args}")
        # invoke your function here and return the
        # output back to the Assistant so it can format
        # the final message
        
        search_results = google_search(function_args["query"], 
                                       google_api_key,
                                       function_args["num_results"])
        return [{ "tool_call_id": tool_call_id,
                 "output": f"Top Five Italian resturants: {search_results}"
               }]

In [12]:
tool_calls = run_1_obj.required_action.submit_tool_outputs.tool_calls
tool_calls

[RequiredActionFunctionToolCall(id='call_NTfY9SxINpIfhc7pt3lJQrz1', function=Function(arguments='{"query":"top 5 Italian restaurants in San Francisco","num_results":5}', name='google_search'), type='function')]

In [13]:
tool_outputs = get_output_for_tool_call(tool_calls)
tool_outputs

Tool ID: call_NTfY9SxINpIfhc7pt3lJQrz1
Function args:{'query': 'top 5 Italian restaurants in San Francisco', 'num_results': 5}


[{'tool_call_id': 'call_NTfY9SxINpIfhc7pt3lJQrz1',
  'output': 'Top Five Italian resturants: [{\'title\': \'15 Primo Italian Restaurants in San Francisco\', \'description\': \'A16Seven HillsCotognaSorellaAcquerelloSPQRPearl 6101Pasta Supply CoBAIAChe FicoFlour + WaterItria\', \'link\': \'https://sf.eater.com/maps/best-italian-restaurants-san-francisco\'}, {\'title\': \'Italian Restaurants in San Francisco\', \'description\': \'Molinari DelicatessenSeven HillsPiccolo FornoFino Bar & RistoranteFrascatiZa PizzaGolden Boy PizzaTraboccoPescatore TrattoriaNob Hill CafeLuna BluThe Italian Homemade Company\', \'link\': \'https://www.tripadvisor.com/Restaurants-g60713-c26-San_Francisco_California.html\'}, {\'title\': \'The 16 Best Italian Restaurants In San Francisco\', \'description\': \'3 days ago — Here are 16 great Italian spots in San Francisco for everything from saucy bolognese to fusilli topped with uni.\', \'link\': \'https://www.theinfatuation.com/san-francisco/guides/best-italian-res

### Step 7: Create the final run to finish.
Submit the output from invoked funcition and pass it back
to the Assistant to generated the final message for the user

In [14]:
run_2_obj = client.beta.threads.runs.submit_tool_outputs(
    thread_id = thread.id,
    run_id = run_1_obj.id,
    tool_outputs=tool_outputs
)

In [15]:
loop_until_completed(client, thread, run_2_obj)

in_progress
completed


In [16]:
print_thread_messages(client, thread)

('assistant:Here are the top five Italian restaurants in San Francisco '
 'according to various sources:\n'
 '\n'
 '1. **A16** - Known for its excellent Southern Italian cuisine and wine, A16 '
 'has received numerous accolades and is often mentioned among the best '
 'Italian spots in San Francisco. [Eater '
 'SF](https://sf.eater.com/maps/best-italian-restaurants-san-francisco)\n'
 '   \n'
 '2. **Seven Hills** - This restaurant has a reputation for delightful Italian '
 "dishes with a focus on pasta. It's a regularly recommended place for Italian "
 'food lovers. [Eater '
 'SF](https://sf.eater.com/maps/best-italian-restaurants-san-francisco), '
 '[TripAdvisor](https://www.tripadvisor.com/Restaurants-g60713-c26-San_Francisco_California.html)\n'
 '\n'
 '3. **Cotogna** - Cotogna offers a range of Italian dishes, including pizzas '
 'and sophisticated entrees, in a rustic and warm setting. [Eater '
 'SF](https://sf.eater.com/maps/best-italian-restaurants-san-francisco)\n'
 '\n'
 '4. **A

In [17]:
# Delete the assistant. 
response = client.beta.assistants.delete(assistant.id)
print(response)

AssistantDeleted(id='asst_LQOXrnwz8i83JEiw8pFoofN8', deleted=True, object='assistant.deleted')
